In [1]:

import os
from queue import SimpleQueue
from get_input import get_input

os.chdir('/Users/soup/Documents/projects/advent-of-code')

In [6]:
input = get_input(2015, 22)

Found a datafile!


In [2]:
test_dict = {'M': {'p_hp': 41, 'p_mana': 447, 'b_hp': 54, 'spent_mana': 53, 'shield': [6, 0], 'regen': [6,9], 'poisoned': [5,2], 'next_options': ["P", 'Q', 'R']}}

In [1]:
sum([1, 3, 5, 11, 13, 17, 19, 23, 31, 37, 41, 43, 47, 53, 67, 97])

508